In [ ]:
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt
%matplotlib inline

# TODO 1 (15%): Implement calc_homography()
def calc_homography(pts_src, pts_dst):
  # enter your code here, and remove the following line
  return np.ones((3,3))


In [ ]:
img_src = cv2.imread("monet.jpg")
img_dst = cv2.imread("painting.jpg")
ht_src, wid_src, _ = img_src.shape
ht_dst, wid_dst, _ = img_dst.shape
pts_src = np.array([    [0,0],[wid_src, 0],[wid_src,ht_src], [0,ht_src]])
pts_dst = np.array([[299,515],  [1776,360],     [1680,1834], [239,1615]])
print('source_shape:', ht_src, wid_src)
print('destination_shape:', ht_dst, wid_dst)

H = calc_homography(pts_src, pts_dst)
H = H/H[2,2]
print(H)

In [ ]:
# TODO 2 (10%): Project the source image on the frame
#         formed by v1, v2, v3, and v4 in the target image via direct mapping.
# Display the projection result.

In [ ]:
# TODO 3 (20%): Project the source image on the frame
#         formed by v1, v2, v3, and v4 in the target image via inverse mapping.
# Display the projection result.

In [ ]:
# TODO 4 (10%): Select key points in library1.jpg and library2.jpg.
# Display the selected points in library1.jpg/library2.jpg as shown in assignment3.pdf
img_dst = cv2.imread("library1.jpg")
img_src = cv2.imread("library2.jpg")
# enter the coordinates of the selected points below (Both pts_dst and pts_src are of shape (4,2).)
# pts_dst = ...
# pts_src = ...
# enter your code here

In [ ]:
# TODO 5 (20%): Warping library2.jpg to create img_warp.
# Display the warped result.

In [ ]:
# TODO 6 (5%): Stitching library1.jpg and img_warped.
# Display the stitched result.

In [ ]:
# TODO 7 (10%): Post-processing the stitched result.
# Display the final result.

In [ ]:
# TODO 8 (10%): Complete the same task as in TODO 3, but without using any for loops.
img_src = cv2.imread("monet.jpg")
img_dst = cv2.imread("painting.jpg")
ht_src, wid_src, _ = img_src.shape
ht_dst, wid_dst, _ = img_dst.shape
pts_src = np.array([    [0,0],[wid_src, 0],[wid_src,ht_src], [0,ht_src]])
pts_dst = np.array([[299,515],  [1776,360],     [1680,1834], [239,1615]])
# enter your code here